# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operaciona` de `Arenales, Armentano, Morabito e Yanass`.</sup>

## Exercício 1

### Descrição do problema
Uma fundição tem de produzir 10 toneladas de um tipo de liga metálica e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínimas e máximas de cada componente da liga).

Composição(%) e custo(R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.005 | 0.9 | 0.09 |
| Silício | 0.14 | - | 0.27 |
| Custo | 90 | 180 | 25 |

Composição(%) mínima e máxima:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 0.095 |
|Silício | 0.19 | 0.2 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip

from mip import *

Carrega Dados

In [3]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14},
    'g': {'c': 0.9,   's': 0.0},
    's': {'c': 0.09,  's': 0.27},
}

# custo
c = {'l': 90, 'g': 180, 's': 25}

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min
m = {'c': 0.095, 's': 0.2} # max

# quantidade desejada da liga
Q = 10

Cria modelo

In [ ]:
model = Model()
x = [model.add_var() for i in range(3)]
c = [90, 180, 25]
model.objective = minimize(xsum(c[i]*x[i] for i in range(3)))

c1 = [0.005, 0.9, 0.09]
c2 = [0.14, 0.27]
model += xsum(c1[i]*x[i] for i in range(3)) <= 0.95, 'maximo de carbono'
model += xsum(c1[i]*x[i] for i in range(3)) >= 0, 'minimo de carbono'

model += xsum([c2[0]*x[0], c2[1]*x[2]]) <= 2, 'maximo de silicio'
model += xsum([c2[0]*x[0], c2[1]*x[2]]) >= 1.9, 'minimo de silicio'

model += xsum(x[i] for i in range(3)) == 10, 'minimo de toneladas'

model += x[0] >= 0, 'nao negatividade de x1'
model += x[1] >= 0, 'nao negatividade de x2'
model += x[2] >= 0, 'nao negatividade de x3'

model.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

Executa

In [14]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")
  
  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
var(0) = 5.38
var(1) = 0.00
var(2) = 4.62


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Código

Carrega Dados

In [15]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

In [16]:
model += x[0] <= 5, 'maximo de toneladas de lingotes'
model += x[1] <= 5, 'maximo de toneladas de grafite'
model += x[2] <= 12, 'maximo de toneladas de sucata'

model.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
obj: 90 var(0) + 180 var(1) + 25 var(2)
Subject To
cons0:  0.00500 var(0) + 0.90000 var(1) + 0.09000 var(2) <= 0.95000
cons1:  0.00500 var(0) + 0.90000 var(1) + 0.09000 var(2) >= -0
cons2:  0.14000 var(0) + 0.27000 var(2) <= 2
cons3:  0.14000 var(0) + 0.27000 var(2) >= 1.90000
cons4:  var(0) + var(1) + var(2) = 10
cons5:  var(0) >= 0
cons6:  var(1) >= 0
cons7:  var(2) >= 0
cons8:  var(0) <= 5
cons9:  var(1) <= 5
cons10:  var(2) <= 12
Bounds
End



Executa

In [17]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
var(0) = 5.00
var(1) = 0.19
var(2) = 4.81


## Exercício 3

Suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e sua composição mínima e máxima é dada na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 0.4 |
|Silício | 0.12 | 0.19 |


### Código

Carrega dados

In [ ]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, {'c': 0.0, 's': 0.12}]
m = [{'c': 0.095, 's': 0.2}, {'c': 0.4, 's': 0.19}]

# quantidade desejada da liga
Q = [10, 6]

Cria modelo

In [34]:
model = Model()
x = [model.add_var() for i in range(6)]
c = [90, 180, 25, 90, 180, 25]
model.objective = minimize(xsum(c[i]*x[i] for i in range(6)))

c1 = [0.005, 0.9, 0.09]
c2 = [0.14, 0.27]
model += xsum(c1[i]*x[i] for i in range(3)) <= 0.95, 'maximo de carbono na liga 1'
model += xsum(c1[i]*x[i] for i in range(3)) >= 0, 'minimo de carbono na liga 1'

model += xsum([c2[0]*x[0], c2[1]*x[2]]) <= 2, 'maximo de silicio na liga 1'
model += xsum([c2[0]*x[0], c2[1]*x[2]]) >= 1.9, 'minimo de silicio na liga 1'

model += xsum(x[i] for i in range(3)) == 10, 'toneladas requeridas da liga 1'

model += x[0] >= 0, 'nao negatividade de x1'
model += x[1] >= 0, 'nao negatividade de x2'
model += x[2] >= 0, 'nao negatividade de x3'
model += x[3] >= 0, 'nao negatividade de x4'
model += x[4] >= 0, 'nao negatividade de x5'
model += x[5] >= 0, 'nao negatividade de x6'

model += xsum([c1[0]*x[3], c1[1]*x[4], c1[2]*x[5]]) <= 2.4, 'maximo de carbono na liga 2'
model += xsum([c1[0]*x[3], c1[1]*x[4], c1[2]*x[5]]) >= 0, 'minimo de carbono na liga 2'

model += xsum([c2[0]*x[3], c2[1]*x[5]]) <= 1.14, 'maximo de silicio na liga 2'
model += xsum([c2[0]*x[3], c2[1]*x[5]]) >= 0.72, 'minimo de silicio na liga 2'

model += xsum(x[i] for i in range(3,6)) == 6, 'toneladas requeridas da liga 2'

model += x[0] + x[3] <= 5, 'maximo de toneladas de lingotes'
model += x[1] + x[4] <= 5, 'maximo de toneladas de grafite'
model += x[2] + x[5] <= 12, 'maximo de toneladas de sucata'

model.write("modelo3.lp") # salva modelo em arquivo
with open("modelo3.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
obj: 90 var(0) + 180 var(1) + 25 var(2) + 90 var(3) + 180 var(4) + 25 var(5)
Subject To
cons0:  0.00500 var(0) + 0.90000 var(1) + 0.09000 var(2) <= 0.95000
cons1:  0.00500 var(0) + 0.90000 var(1) + 0.09000 var(2) >= -0
cons2:  0.14000 var(0) + 0.27000 var(2) <= 2
cons3:  0.14000 var(0) + 0.27000 var(2) >= 1.90000
cons4:  var(0) + var(1) + var(2) = 10
cons5:  var(0) >= 0
cons6:  var(1) >= 0
cons7:  var(2) >= 0
cons8:  var(3) >= 0
cons9:  var(4) >= 0
cons10:  var(5) >= 0
cons11:  0.00500 var(3) + 0.90000 var(4) + 0.09000 var(5) <= 2.40000
cons12:  0.00500 var(3) + 0.90000 var(4) + 0.09000 var(5) >= -0
cons13:  0.14000 var(3) + 0.27000 var(5) <= 1.14000
cons14:  0.14000 var(3) + 0.27000 var(5) >= 0.72000
cons15:  var(3) + var(4) + var(5) = 6
cons16:  var(0) + var(3) <= 5
cons17:  var(1) + var(4) <= 5
cons18:  var(2) + var(5) <= 12
Bounds
End



Executa

In [35]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 1029.26

Solution:
var(0) = 4.32
var(1) = 0.51
var(2) = 5.17
var(3) = 0.68
var(4) = 1.45
var(5) = 3.87
